In [18]:
rules={
    ('S','NP','VP'):1,
    ('PP','P','NP'):1,
    ('VP','V','NP'):0.7,
    ('VP','VP','PP'):0.3,
    ('P','with'):1,
    ('V','saw'):1,
    ('NP','NP','PP'):0.4,
    ('NP','astronomers'):0.1,
    ('NP','ears'):0.18,
    ('NP','saw'):0.04,
    ('NP','stars'):0.18,
    ('NP','telescopoes'):0.1,
}

In [19]:
from nltk import word_tokenize
sentence = 'astronomers saw stars with ears'
words = word_tokenize(sentence)
n = len(words)

In [26]:
table = [
    [{},{},{},{},{}],
    [{},{},{},{},{}],
    [{},{},{},{},{}],
    [{},{},{},{},{}],
    [{},{},{},{},{}],
]

In [27]:
for i in range(n):
    for r in rules.keys():
        if words[i] in r:
            table[i][i][r[0]] = rules[r]
            
table

[[{'NP': 0.1}, {}, {}, {}, {}],
 [{}, {'V': 1, 'NP': 0.04}, {}, {}, {}],
 [{}, {}, {'NP': 0.18}, {}, {}],
 [{}, {}, {}, {'P': 1}, {}],
 [{}, {}, {}, {}, {'NP': 0.18}]]

In [28]:
for i in range(n):
    p=0
    q=p+i+1
    while p<n and q<n:
        for r in range(p,q):
            dj = table[p][r]
            dk = table[r+1][q]
            for N in rules.keys():
                if N[-2] in dj.keys() and N[-1] in dk.keys():
                    val = rules[N]*dj[N[-2]]*dk[N[-1]]
                    if N[0] in table[p][q]:
                        table[p][q][N[0]] +=val
                    else:
                        table[p][q][N[0]] = val
        p+=1
        q = p+i+1
table

[[{'NP': 0.1}, {}, {'S': 0.0126}, {}, {'S': 0.0015875999999999998}],
 [{}, {'V': 1, 'NP': 0.04}, {'VP': 0.126}, {}, {'VP': 0.015875999999999998}],
 [{}, {}, {'NP': 0.18}, {}, {'NP': 0.01296}],
 [{}, {}, {}, {'P': 1}, {'PP': 0.18}],
 [{}, {}, {}, {}, {'NP': 0.18}]]

In [32]:
import nltk
grammer = nltk.PCFG.fromstring("""
S -> NP VP [1.0]
PP -> P NP [1.0]
VP -> V NP [0.7] | VP PP [0.3]
P -> 'with' [1.0]
V -> 'saw' [1.0]
NP -> NP PP [0.4] | 'astronomers' [0.1] | 'ears' [0.18] | 'saw' [0.04] | 'stars' [0.18] | 'telescopes' [0.1] 
""")
rd_parser = nltk.InsideChartParser(grammer)
for tree in rd_parser.parse(words):
    print(tree)
    tree.pretty_print()

(S
  (NP astronomers)
  (VP (V saw) (NP (NP stars) (PP (P with) (NP ears))))) (p=0.0009072)
                  S                    
      ____________|____                 
     |                 VP              
     |        _________|___             
     |       |             NP          
     |       |     ________|____        
     |       |    |             PP     
     |       |    |         ____|___    
     NP      V    NP       P        NP 
     |       |    |        |        |   
astronomers saw stars     with     ears

(S
  (NP astronomers)
  (VP (VP (V saw) (NP stars)) (PP (P with) (NP ears)))) (p=0.0006804)
                      S                    
      ________________|____                 
     |                     VP              
     |            _________|________        
     |           VP                 PP     
     |        ___|____          ____|___    
     NP      V        NP       P        NP 
     |       |        |        |        |   
astronomers sa